In [2]:
# quco imports
from qucochemistry.vqe import VQEexperiment

# Rigetti imports
from grove.pyvqe.vqe import VQE

# pyquil imports
from pyquil import Program
from pyquil.gates import *
import pyquil.api as api
from pyquil.paulis import *
import numpy as np
from scipy.optimize import minimize
import math

In [3]:
def U(n):
    # Builds the interaction term
    # a_p^dagger a_q^dagger a_q a_p
    U = 2
    
    hamiltonian = ZERO()
    for i in range(n):
        hamiltonian += (sI() - sZ(i) - sZ(i+n) + sZ(i)*sZ(i+n))
    hamiltonian *= (U / 4)
        
    return hamiltonian

In [4]:
def e(n):
    # Builds the number term
    # a_p^dagger a_p
    e = -1.5
    hamiltonian = ZERO()
    for i in range(n):
        hamiltonian += (2*sI() - sZ(i) - sZ(i+n))
    hamiltonian *= (e / 2)
    
    return hamiltonian

In [5]:
def t(n):
    # Builds the hopping term
    # Sites are organized like 0 1 2 3 | 4 5 6 7, where first N are up and last N are down
    # overall structure is a ring, so only two hoppings per site are possible
    # hopping includes periodic bounding conditions
    # a_p^dagger a_q a_q^dagger a_p
    t = 1
    
    hamiltonian = ZERO()
    
    
    return hamiltonian

In [7]:
# hopping term: a_p^dagger a_q
t = 1
hamt = (sX(0)*sX(1) + sY(0)*sY(1) + sX(0)*sZ(1)*sZ(2)*sX(3) + sY(0)*sZ(1)*sZ(2)*sY(3) + sX(1)*sX(2) + sY(1)*sY(2) + sX(2)*sX(3) + sY(2)*sY(3))
hamt += (sX(4)*sX(5) + sY(4)*sY(5) + sX(4)*sZ(5)*sZ(6)*sX(7) + sY(4)*sZ(5)*sZ(6)*sY(7) + sX(5)*sX(6) + sY(5)*sY(6) + sX(6)*sX(7) + sY(6)*sY(7))
hamt *= (-t / 2)

# interaction term: a_p^dagger a_q^dagger a_q a_p
hamU = U(4)
hame = e(4)

hamiltonian = hamU + hamt
print(hamiltonian)

(2+0j)*I + (-0.5+0j)*Z0 + (-0.5+0j)*Z4 + (0.5+0j)*Z0*Z4 + (-0.5+0j)*Z1 + (-0.5+0j)*Z5 + (0.5+0j)*Z1*Z5 + (-0.5+0j)*Z2 + (-0.5+0j)*Z6 + (0.5+0j)*Z2*Z6 + (-0.5+0j)*Z3 + (-0.5+0j)*Z7 + (0.5+0j)*Z3*Z7 + (-0.5+0j)*X0*X1 + (-0.5+0j)*Y0*Y1 + (-0.5+0j)*X0*Z1*Z2*X3 + (-0.5+0j)*Y0*Z1*Z2*Y3 + (-0.5+0j)*X1*X2 + (-0.5+0j)*Y1*Y2 + (-0.5+0j)*X2*X3 + (-0.5+0j)*Y2*Y3 + (-0.5+0j)*X4*X5 + (-0.5+0j)*Y4*Y5 + (-0.5+0j)*X4*Z5*Z6*X7 + (-0.5+0j)*Y4*Z5*Z6*Y7 + (-0.5+0j)*X5*X6 + (-0.5+0j)*Y5*Y6 + (-0.5+0j)*X6*X7 + (-0.5+0j)*Y6*Y7


In [251]:
vqe = VQEexperiment(hamiltonian=hamiltonian, method='WFS', optimizer='Nelder-Mead', strategy='custom_program', parametric=True, tomography = True, shotN=100000, verbose=False)
ansatz = var_ansatz()
vqe.set_custom_ansatz(ansatz)
print('Exact ground state energy:',vqe.get_exact_gs())
print('empty circuit energy estimate: ', vqe.objective_function())

Exact ground state energy: -8.828427124746186
empty circuit energy estimate:  -5.00357


In [5]:
from pyquil.quil import DefGate
from pyquil.quilatom import Parameter, quil_exp

yb = np.array([[1/math.sqrt(2)+0j, complex(0, 1/math.sqrt(2))],
               [complex(0, 1/math.sqrt(2)), 1/math.sqrt(2)+0j]])
yb_definition = DefGate("YB", yb)
YB = yb_definition.get_constructor()

ybd = yb.conj().T
ybd_definition = DefGate("YBD", ybd)
YBD = ybd_definition.get_constructor()

theta = Parameter('theta')
zr = np.array([[quil_exp(1j * theta/2), 0],
               [0, quil_exp(1j * -theta/2)]])
zr_definition = DefGate('ZR', zr, [theta])
ZR = zr_definition.get_constructor()

print(yb)
print(ybd)

[[0.70710678+0.j         0.        +0.70710678j]
 [0.        +0.70710678j 0.70710678+0.j        ]]
[[0.70710678-0.j         0.        -0.70710678j]
 [0.        -0.70710678j 0.70710678-0.j        ]]


In [6]:
def var_ansatz():
    S = 2
    p = Program()
    
    # add new YB and YBD and ZR gate
    p += yb_definition
    p += ybd_definition
    p += zr_definition
        
    # declare theta in memory
    params = p.declare('theta', memory_type='REAL', memory_size=S*2)
    
    p.inst(H(0))
    p.inst(H(2))
    p.inst(CNOT(0, 1))
    p.inst(CNOT(2, 3))
    p.inst(X(0))
    p.inst(X(2))
    
    U1 = 0
    U2 = 1
    
    for i in range(S):
        # building U_u
        p.inst(RZ(U1*params[2*i]/4, 0))
        p.inst(RZ(U1*params[2*i]/4, 1))
        p.inst(RZ(U2*params[2*i]/4, 2))
        p.inst(RZ(U2*params[2*i]/4, 3))
        p.inst(CNOT(0, 2))
        p.inst(RZ(-U1*params[2*i]/4, 2))
        p.inst(CNOT(0, 2))
        p.inst(CNOT(1, 3))
        p.inst(RZ(-U2*params[2*i]/4, 3))
        p.inst(CNOT(1, 3))
        
        # building U_t
        p.inst(H(0))
        p.inst(H(1))
        p.inst(H(2))
        p.inst(H(3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(RZ(params[2*i+1], 1))
        p.inst(RZ(params[2*i+1], 3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(H(0))
        p.inst(H(1))
        p.inst(H(2))
        p.inst(H(3))
        p.inst(RX(-np.pi/2, 0))
        p.inst(RX(-np.pi/2, 1))
        p.inst(RX(-np.pi/2, 2))
        p.inst(RX(-np.pi/2, 3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(RZ(params[2*i+1], 1))
        p.inst(RZ(params[2*i+1], 3))
        p.inst(CNOT(0, 1))
        p.inst(CNOT(2, 3))
        p.inst(RX(-np.pi/2, 0))
        p.inst(RX(-np.pi/2, 1))
        p.inst(RX(-np.pi/2, 2))
        p.inst(RX(-np.pi/2, 3))
        
        # building U_u
        p.inst(RZ(U1*params[2*i]/4, 0))
        p.inst(RZ(U1*params[2*i]/4, 1))
        p.inst(RZ(U2*params[2*i]/4, 2))
        p.inst(RZ(U2*params[2*i]/4, 3))
        p.inst(CNOT(0, 2))
        p.inst(RZ(-U1*params[2*i]/4, 2))
        p.inst(CNOT(0, 2))
        p.inst(CNOT(1, 3))
        p.inst(RZ(-U2*params[2*i]/4, 3))
        p.inst(CNOT(1, 3))
    return p

In [159]:
initial_angles = np.random.rand(4)
result = vqe.start_vqe(initial_angles, maxiter=50)

0: [0.75770986 0.35881646 0.42018572 0.3099097 ]
1: [0.77875735 0.36854341 0.37588903 0.31831087]
2: [0.77875735 0.36854341 0.37588903 0.31831087]
3: [0.67615081 0.40542477 0.38933624 0.30909292]
4: [0.55183904 0.40255397 0.37758971 0.34675231]
5: [0.55183904 0.40255397 0.37758971 0.34675231]
6: [0.4409286  0.44944514 0.32371683 0.34092911]
7: [0.37059214 0.46797236 0.34455258 0.34905498]
8: [0.32041935 0.4623648  0.30871355 0.37825395]
9: [0.32041935 0.4623648  0.30871355 0.37825395]
10: [0.32041935 0.4623648  0.30871355 0.37825395]
11: [0.32041935 0.4623648  0.30871355 0.37825395]
12: [0.32041935 0.4623648  0.30871355 0.37825395]
13: [0.30672885 0.47175953 0.30874061 0.36059431]
14: [0.20091175 0.49868087 0.29585396 0.36987469]
15: [0.20091175 0.49868087 0.29585396 0.36987469]
16: [0.24410325 0.4889867  0.31024337 0.36597179]
17: [0.24410325 0.4889867  0.31024337 0.36597179]
18: [0.24410325 0.4889867  0.31024337 0.36597179]
19: [0.24410325 0.4889867  0.31024337 0.36597179]
20: [0.244

In [160]:
result

 final_simplex: (array([[0.24410325, 0.4889867 , 0.31024337, 0.36597179],
       [0.24411505, 0.48898772, 0.31023858, 0.3659705 ],
       [0.24409658, 0.48898662, 0.31024408, 0.36597111],
       [0.24410313, 0.48899191, 0.31023466, 0.36596976],
       [0.24410653, 0.48898887, 0.31023671, 0.36597115]]), array([-1.7684 , -1.76593, -1.76563, -1.76396, -1.76244]))
           fun: -1.7684000000000002
       message: 'Maximum number of iterations has been exceeded.'
          nfev: 128
           nit: 50
        status: 2
       success: False
             x: array([0.24410325, 0.4889867 , 0.31024337, 0.36597179])

In [9]:
def op(s1, s2):
    # returns a PauliSum representing a_s1^daggar a_s2
    def sigma_plus(s):
        # returns sigma^+ operator on side s
        # sigma^+ = 1/2(X + iY)
        return PauliTerm('X', s, 0.5) + PauliTerm('Y', s, complex(0.0+0.5j))
    def sigma_minus(s):
        # returns sigma^- operator on side s
        # sigma^- = 1/2(X - iY)
        return PauliTerm('X', s, 0.5) - PauliTerm('Y', s, complex(0.0+0.5j))
    
    # for all sites in between site one (s1) and site two (s2), multiply by sigma z
    z = sI()
    for i in range(s1+1, s2):
        z *= PauliTerm('Z', i)
        
    return sigma_minus(s1) * z * sigma_plus(s2)

In [195]:
# building the single particle density matrix (spdm) 
spdm = np.zeros((16,16))
for i in range(16):
    #for j in range(16):
    # building an operator and vqe for each i, j
    partial_vqe = VQEexperiment(hamiltonian=op(i, i), method='WFS', optimizer='Nelder-Mead', strategy='custom_program', parametric=True, tomography = True, shotN=100000, verbose=False)
    ansatz = var_ansatz()
    partial_vqe.set_custom_ansatz(ansatz)

    # element (j, i) in the spdm is a_i^dagger a_j
    spdm[i, i] = partial_vqe.objective_function(result.x)

In [196]:
np.set_printoptions(suppress=True)
print(np.around(spdm, decimals=2))

[[0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.5 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  

In [197]:
np.trace(spdm)

1.99607